In this project you are trying out for an online marketplace company. The evaluation cover 3 important aspects of the daily job there that you need to know in order to get the position. 

The name of the database you will be working with is: data-analyst-web-store



Step 1:

Queries to know the data:

1. How many customers are there and what is the gender ratio?

2. How many items are there? Do you see any phenomena? (notice that item_id can be different)

3. What is the total amount of orders?

4. How many users were registered in July – December 2019 on the YouTube campaign?

5. How many users from the US bought the book Rich Dad? What is their share out of everyone who bought this book? And what is their share out of all US sales?

6. What is the most ordered item by quantity and what is the item with the highest total revenue? 

7. What is the share of total revenue, orders and users registration for each country, And how this ratio changes from 2019 to 2020 (show results for both years)

8. What is the total of shipments for each category in the data?

9. What is the average of the first order of each customer?

10. Does the membership split equally between gender? Do we get the same result per different countries in the data? Check only for English speaking countries (US, CA and EN)

11. What is the most successful campaign in terms of registrations? And what is the most successful campaign in terms of total revenue brought in?


In [2]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [3]:
db_config = {'user': 'practicum_student',         # username             
                      'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password             
                      'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',       #host URL      
                      'port': 6432,              # connection port             
                      'db': 'data-analyst-web-store'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [4]:
def queryResult(q):
    return pd.io.sql.read_sql(q, con = engine)

In [9]:
query = '''
            select gender,count(distinct customer_id)
            from customer_info
            group by gender
         '''
display(queryResult(query1))

,gender,count
0,Female,241
1,Male,218




1. How many customers are there and what is the gender ratio?



In [6]:

query1 = '''
            select
                sum(case when "gender" = 'Male' then 1 else 0 end)/count(*)::float as male_ratio,
                sum(case when "gender" = 'Female' then 1 else 0 end)/count(*)::float as female_ratio,
                sum(customer_id) as total
            from 
                customer_info
         '''
display(queryResult(query1))

,male_ratio,female_ratio,total
0,0.477083,0.522917,117859


2. How many items are there? Do you see any phenomena? (notice that item_id can be different)



In [7]:
query2 = '''
             select count(*),count(distinct item_id) as item_count, count(distinct item) as item_name_count  
             from items
         '''
display(queryResult(query1))

,count,item_count,item_name_count
0,175,175,137


We can see clearly that the item name is not unique and it can appear multiple time even the id or the price will be different

3. What is the total amount of orders?

In [8]:
query3 = '''
            select sum(amount) as total_orders
            from order_details
         '''
display(queryResult(query1))

,total_orders
0,3180


4. How many users were registered in July – December 2019 on the YouTube campaign?

In [9]:
query4 = '''
             select count(distinct customer_info.customer_id) 
             from campaign inner join registration on registration.campaign_id = campaign.campaign_id
                           inner join customer_info on registration.customer_id = customer_info.customer_id
             where cast(registration."Registration" as date) between '01-06-2019' AND '31-12-2019'
                   AND campaign."source" = 'YouTube' 
         '''
display(queryResult(query1))

,count
0,44


5. How many users from the US bought the book Rich Dad? What is their share out of everyone who bought this book? And what is their share out of all US sales?

In [10]:
query5 = '''
        select sum(amount) as sum_US_total_sales_Rich_Dad
        from customer_info left join countries on countries.country_id = customer_info.country_id
                           left join orders on customer_info.customer_id = orders.customer_id
                           left join order_details on orders.order_id = order_details.order_id
                           left join items on order_details.item_id = items.item_id
        where "item" = 'Rich Dad' and "symbol" = 'US'
         '''
# where "item" = 'Rich Dad'
display(queryResult(query1))

,sum_us_total_sales_rich_dad
0,2


6. What is the most ordered item by quantity and what is the item with the highest total revenue? 

In [11]:
query6 = ''' select quant."item id by quantity",quant."quantity",rev."item id by revenue",rev."revenue"
             from (select items.item_id as "item id by quantity",count(order_details.order_id * order_details.amount) as "quantity"
                   from order_details inner join items on items.item_id=order_details.item_id
                   group by "item id by quantity"
                   order by "quantity" desc) as quant,
                  (select items.item_id as "item id by revenue",sum(items.price * order_details.amount) as "revenue"
                   from order_details inner join items on items.item_id=order_details.item_id
                   group by "item id by revenue"
                   order by "revenue" desc) as rev
              limit 1     
         '''
display(queryResult(query1))


,item id by quantity,quantity,item id by revenue,revenue
0,58,20,152,22319.76



7. What is the share of total revenue, orders and users registration for each country, And how this ratio changes from 2019 to 2020 (show results for both years)


In [12]:
query7 = '''
            select
                country,
                year,
                revenue / sum(revenue) over (partition by year) *100 as revenue_ratio,
                order_cnt / sum(order_cnt) over (partition by year) *100 as ordr_ratio,
                customer_cnt / sum(customer_cnt) over (partition by year) *100 as registration_ratio
            from
                (select 
                        country,
                        sum(price * amount) as revenue,
                        count(distinct orders.order_id) as order_cnt,
                        count(distinct customer_info.customer_id) as customer_cnt,
                        extract(year from orders.date) :: int as year
                from customer_info left join countries on countries.country_id = customer_info.country_id
                                   left join orders on customer_info.customer_id = orders.customer_id
                                   left join order_details on orders.order_id = order_details.order_id
                                   left join items on order_details.item_id = items.item_id 
                                   left join registration on registration.customer_id = customer_info.customer_id
                where extract(year from date) > 2018 and extract(year from date) < 2021
                group by country, year
                ) as final_table
        order by country
         '''


display(queryResult(query1))

,country,year,revenue_ratio,ordr_ratio,registration_ratio
0,Belguim,2019,1.394121,3.797468,3.759398
1,Belguim,2020,6.668262,6.562500,7.027027
2,Canada,2019,17.109629,14.556962,15.789474
3,Canada,2020,9.139087,13.906250,13.513514
4,England,2020,8.048886,5.781250,5.135135
5,England,2019,3.904886,7.594937,6.766917
6,France,2020,11.208516,10.625000,10.000000
7,France,2019,16.037476,10.126582,10.526316
8,Germany,2019,7.728985,6.962025,7.518797
9,Germany,2020,11.706142,8.750000,8.648649



8. What is the total of shipments for each category in the data?


In [13]:
query8 = '''select categories.category_id, categories.category, 
                   sum(case when "country" = 'US' then "amount"*"us_ship" else 0 end)::float +
                   sum(case when "country" not in ('US') then "amount"*"other_ship" else 0 end)::float as "Total shipment"  
            from countries inner join customer_info on countries.country_id = customer_info.country_id
                           inner join orders on customer_info.customer_id = orders.customer_id
                           inner join order_details on orders.order_id = order_details.order_id
                           inner join items on order_details.item_id = items.item_id
                           inner join categories on items.category_id = categories.category_id
            group by categories.category_id, categories.category
         '''
# sum(us_ship+other_ship) as "total shipments"
display(queryResult(query1))

,category_id,category,Total shipment
0,9,Gaming,3567.62
1,5,Sporting Goods,7516.24
2,3,Books,6970.35
3,7,Home & Garden,5247.90
4,4,Collectibles,8094.60
5,2,Fashion,7674.88
6,8,Kitchen,3432.71
7,10,Beauty Care,3018.49
8,1,Electronics,6872.25
9,6,Toys & Hobbies,3336.66



9. What is the average of the first order of each customer?


In [14]:
query9 = '''
            select customer_info."customer_id","date",avg("amount"*"price")
            from customer_info inner join orders on customer_info.customer_id = orders.customer_id
                               inner join order_details on orders.order_id = order_details.order_id
                               inner join items on order_details.item_id = items.item_id
            group by customer_info."customer_id","date"
            having "date"=min("date")
         '''
display(queryResult(query1))

,customer_id,date,avg
0,409,2021-03-23,375.000
1,438,2020-01-08,59.970
2,217,2019-11-17,49.485
3,430,2019-12-12,179.970
4,167,2021-04-30,208.980
...,...,...,...
1192,139,2020-07-24,283.980
1193,172,2020-10-15,705.000
1194,81,2020-03-30,9.740
1195,435,2021-03-07,449.970



10. Does the membership split equally between gender? Do we get the same result per different countries in the data? Check only for English speaking countries (US, CA and EN)


In [15]:
query10 = '''select "country",
                    100*sum(case when "gender" = 'Male' then 1 else 0 end)/count(*)::float as male_ratio,
                    100*sum(case when "gender" = 'Female' then 1 else 0 end)/count(*)::float as female_ratio 
             from customer_info inner join countries on countries.country_id = customer_info.country_id
             where "symbol" in ('US','CA','EN')
             group by "country"
          '''

display(queryResult(query1))


,country,male_ratio,female_ratio
0,England,56.00000,44.00000
1,Canada,50.00000,50.00000
2,United States,48.93617,51.06383



11. What is the most successful campaign in terms of registrations? And what is the most successful campaign in terms of total revenue brought in?


In [16]:
query11 = '''
            select campaign_id, camp_rev, mem_cnt 
            from
                (select  campaign_id, sum(items.price * cst_order_details.amount) as camp_rev, count(case when "membership" = 'yes' then 1 else 0 end) as mem_cnt
                from items left join 
                    ( select order_details.item_id, order_details.amount, order_details.order_id, cst_orders.campaign_id,cst_orders.membership
                    from order_details left join 
                        ( select orders.order_id, orders.customer_id, cst_info.campaign_id,  cst_info.membership
                        from orders left join 
                            ( select registration.campaign_id, registration.customer_id, customer_info.membership
                            from customer_info left join registration on registration.customer_id = customer_info.customer_id
                            ) as cst_info on cst_info.customer_id = orders.customer_id
                        ) as cst_orders on cst_orders.order_id = order_details.order_id
                    ) as cst_order_details on cst_order_details.item_id = items.item_id
                    group by campaign_id) final_Table
                order by camp_rev desc, mem_cnt desc
                limit 2

         '''
display(queryResult(query1))

,campaign_id,camp_rev,mem_cnt
0,2,64520.39,477
1,3,58216.37,354
